# Car Detection

Import Libraries

In [107]:
import cv2
import numpy as np

Function for Selecting Region of Interest by dragging mouse

In [108]:
def select_roi(event, x, y, flags, param):
    global selecting_roi, roi_selected, roi

    if event == cv2.EVENT_LBUTTONDOWN:
        roi = (x, y)
        selecting_roi = True
    elif event == cv2.EVENT_MOUSEMOVE:
        if selecting_roi:
            roi = (roi[0], roi[1], x - roi[0], y - roi[1])
    elif event == cv2.EVENT_LBUTTONUP:
        roi = (roi[0], roi[1], x - roi[0], y - roi[1])
        selecting_roi = False
        roi_selected = True

Path to video

In [109]:
video_path = r'[INSER PATH HERE]'

Video detection parameters

In [110]:
#Minimum HSV Change
hue_threshold = 100
saturation_threshold = 140
value_threshold = 30

#Size of Object
min_contour_area = 1000
max_contour_area = 50000

#Noise Removal Parameter
erosion_kernel = np.ones((1, 1), np.uint8)
erosion_iteration = 1
opening_kernel = np.ones((1, 1), np.uint8)

Initial window to choose Region of Interest

In [111]:
#Initialize the video window with the file path
cap = cv2.VideoCapture(video_path)

#Object to check if Region of Interest window is still being chosen
selecting_roi = False
roi_selected = False
roi = None

#Capture the first frame of the video for Region of Interest window
ret, frame = cap.read()

# Convert the first frame to HSV
hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

# Extract the background hue from the first frame
background_hue = hsv_frame[:, :, 0]

#Create selection window
cv2.namedWindow('Select ROI')

#Set mouse to be use as a selector using the function
cv2.setMouseCallback('Select ROI', select_roi)

#Close the window if Region of Interest is chosen/'q' pressed
while not roi_selected:
    cv2.imshow('Select ROI', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyWindow('Select ROI')

Detection process

In [112]:
#Background subtraction using CV2
fgbg = cv2.createBackgroundSubtractorMOG2()

#Initialize Car Count/Detected
object_detected = False
object_count = 0
object_passed = False

#Initialize centroid of object detection
object_centroid = None

#Detection Process
while True:
    #Read a frame from the video
    ret, frame = cap.read()
    
    #Break if the frame is not available
    if not ret:
        break
        
    #Draw a square border around the Region of Interest
    cv2.rectangle(frame, (roi[0], roi[1]), (roi[0] + roi[2], roi[1] + roi[3]), (255, 0, 0), 2)
    
    #Display the object count
    cv2.putText(frame, f'Vehicles passed: {object_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    #Aply the background subtractor to detect moving objects
    fgmask = fgbg.apply(frame)
    
    #Covert the frame into HSV
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    #Calculate the absolute difference to see changes
    diff = cv2.absdiff(hsv_frame[:,:,0], background_hue)
    
    #HSV tolerance detection
    hue_mask = cv2.inRange(diff, 0, hue_threshold)
    saturation_mask = cv2.inRange(hsv_frame[:,:,1], 0, saturation_threshold)
    value_mask = cv2.inRange(hsv_frame[:,:,2], 0, value_threshold)
    
    #Combine all mask
    mask = cv2.bitwise_or(cv2.bitwise_or(hue_mask, saturation_mask), value_mask)
    
    #Remove background noises
    mask = cv2.erode(mask, erosion_kernel, iterations = erosion_iteration)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, opening_kernel)
    
    #Apply the mask to the foreground
    fgmask = cv2.bitwise_and(fgmask, mask)
    
    #Find contours in the mask
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #Draw square in contour with appropriate area if it is in Region of Interest
    for contour in contours:
        area = cv2.contourArea(contour)
        if min_contour_area < area < max_contour_area:
            x, y, w, h = cv2.boundingRect(contour)
            if x > roi[0] and y > roi[1] and x + w < roi[0] + roi[2] and y + h < roi[1] + roi[3]:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                #Create a centriod in the marked rectangle
                centroid_x = x + w // 2
                centroid_y = y + h // 2
                object_centroid = (centroid_x, centroid_y)
                #Visualize the centroid
                cv2.circle(frame, object_centroid, 10, (255, 0, 255), -1)

    #Check if the object centriod passed through the Region of Interest
    if object_centroid is not None and object_centroid[1] < roi[1] + roi[3] // 2 and not object_passed:
        object_count += 1
        object_passed = True
    elif object_centroid is not None and object_centroid[1] > roi[1] + roi[3] // 2:
        object_passed = False
    
    #Resize detection frame and the video to fit the window
    resized_frame = cv2.resize(frame, (frame.shape[1] // 2, frame.shape[0] // 2))
    resized_fgmask = cv2.resize(fgmask, (fgmask.shape[1] // 2, fgmask.shape[0] // 2))
    _, binary_fgmask = cv2.threshold(resized_fgmask, 1, 255, cv2.THRESH_BINARY)
    binary_fgmask = cv2.cvtColor(binary_fgmask, cv2.COLOR_GRAY2BGR)
    output_frame = np.hstack((resized_frame, binary_fgmask))

    #Display the results
    cv2.imshow('Original vs Detected Objects', output_frame)
    
    #Stop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#Close window and end code
cap.release()
cv2.destroyAllWindows()